# 131. The Skyline Problem
#### https://www.lintcode.com/problem/the-skyline-problem

# Approach 1

In [25]:
class Solution:
    """
    @param buildings: A list of lists of integers
    @return: Find the outline of those buildings
    """
    def buildingOutline(self, buildings):
        # write your code here
        points = []
        for i, building in enumerate(buildings):
            start, end, height = building
            points.append((start, height, i, True))
            points.append((end, height, i, False))
        points.sort()
        
        maxheap = MyHeapHash()
        res = []
        pre_position = None
        for position, height, index, is_start in points:
            max_h = 0
            if maxheap.size: max_h = -maxheap.top()[0]
            self.merge(res, pre_position, position, max_h)
            if is_start:
                maxheap.push((-height, index))
            else:
                maxheap.remove((-height, index))
            pre_position = position
            
        return res
            
    def merge(self, res, start, end, height):
        if start == None or height == 0 or start == end: return
        if res == []:
            res.append([start, end, height])
            return

        if start == res[-1][1] and height == res[-1][2]:
            res[-1][1] = end
            return
        
        res.append([start, end, height])

# Verification

In [26]:
testcases = [[[1,3,3],[2,4,4],[5,6,1]],[[1,4,3],[6,9,5]]]
expected = [[[1,2,3],[2,4,4],[5,6,1]],[[1,4,3],[6,9,5]]]

outputs = []
sol = Solution();

for case in zip(testcases, expected):
    print("Input:    " + str(case[0]))
    print("Output:   " + str(sol.buildingOutline(case[0])))
    print("Expected: " + str(case[1]) + "\n")

Input:    [[1, 3, 3], [2, 4, 4], [5, 6, 1]]
[(1, 3, 0, True), (2, 4, 1, True), (3, 3, 0, False), (4, 4, 1, False), (5, 1, 2, True), (6, 1, 2, False)]
Output:   [[1, 2, 3], [2, 4, 4], [5, 6, 1]]
Expected: [[1, 2, 3], [2, 4, 4], [5, 6, 1]]

Input:    [[1, 4, 3], [6, 9, 5]]
[(1, 3, 0, True), (4, 3, 0, False), (6, 5, 1, True), (9, 5, 1, False)]
Output:   [[1, 4, 3], [6, 9, 5]]
Expected: [[1, 4, 3], [6, 9, 5]]



# My Heap Hash

In [14]:
class MyHeapHash(object):

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.heap = []
        self.hash = {}
        self.size = 0
        
    def top(self):
        return self.heap[0]
    
    def push(self, key):
        """
        :type key: int
        :rtype: None
        """
        #if key in self.hash: return
        self.heap.append(key)
        self.hash[key] = len(self.heap) - 1
        self.size += 1
        self.sift_up(self.hash[key])

    def remove(self, key):
        """
        :type key: int
        :rtype: None
        """
        if key not in self.hash: return
        index = self.hash[key]
            
        self.heap[index], self.heap[self.size - 1] = self.heap[self.size - 1], self.heap[index]
        
        self.hash[self.heap[index]] = index

        self.heap.pop()
        del self.hash[key]
        self.size -= 1
        
        if index < self.size:
            self.sift_up(index)
            self.sift_down(index)
            
    def pop(self):
        val = self.heap[0]
        self.remove(val)
        return val
    
    def sift_up(self, index):   
        while (index - 1) // 2 >= 0:
            if self.heap[index] >=  self.heap[(index - 1) // 2]: break
            val1, val2 = self.heap[index], self.heap[(index - 1) // 2]
            self.heap[index], self.heap[(index - 1) // 2] = val2, val1
            self.hash[val1], self.hash[val2] = (index - 1) // 2, index
            index = (index - 1) // 2
            
    def sift_down(self, index):
        while index * 2 + 1 < self.size:
            left = right = index * 2 + 1
            if right < self.size - 1: right += 1
            if self.heap[index] <= min(self.heap[left], self.heap[right]): break
            target = right
            if self.heap[left] < self.heap[right]: target = left
            val1, val2 = self.heap[index], self.heap[target]
            self.heap[index], self.heap[target] = val2, val1
            self.hash[val1], self.hash[val2] = target, index
            index = target